## Scikit-Learn (sklearn) Course

<span>
0. sklearn workflow overview<br>
1. preparing data (collecting, exploring, cleaning, transforming, reducing, splitting)<br>
2. defining problem / selecting machine learning model<br>
3. training model and making predictions<br>
<span style="color:orange">4. evaluating model</span><br>
5. improving model<br>
6. saving and loading model<br>
7. putting it all together
</span>

## 4. Evaluating Model

#### General concepts

--- resources  
[sklearn documentation > model evaluation](https://scikit-learn.org/stable/modules/model_evaluation.html)  
[statquest youtube video: ROC and AUC explained](https://www.youtube.com/watch?v=4jRBRDbJemM)  
[sklearn documentation > ROC curve for multiclass classification models](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html)  
<span style="color:red">>>> One-vs-Rest multiclass ROC<span>

--- sklearn built-in evaluation methods  
`.score()` method  
cross valiadion  
metric functions

--- cross validation  
creates `cv=k` different train/test splits from the same dataset (k-fold cross validation)  
trains and scores the algorithm on all splits > training covers the entire dataset  
scoring metric is defined by the `scoring=` parameter  
`scoring=None` invokes the default scorer

--- area under the ROC curve (AUC)  
integral of ROC curve > ranges between 0.0-1.0  
used to compare the performance of different algorithms

--- confusion matrix  
two-dimensional array of targets (rows) vs predictions (columns)  
a quick way to compare targets to predictions  
gives an idea of where the model is confused

--- classification report  
a summary table of several classification metrics  
**precision:** true within-class predictions / all within-class predictions  
**recall:** true within-class predictions / all within-class targets  
**f1-score:** within-class mean of precision and recall  
**support:** all within-class targets  
**accuracy:** true predictions / all predictions  
**macro average:** accross-class means of precision, recall, and f1-score  
**weighted average:** support-weighted accross-class means of precision, recall, and f1-score

--- regression model metrics  
coefficient of determination (R^2)  
mean absolute error (MAE)  
mean squared error (MSE)

--- coding tricks within jupyter notebook  
**`!command`, e.g., `!dir`** runs terminal command within jupyter notebook  
**`sklearn.__version__`** displays version of installed module

#### Creating classification model

In [ ]:
### imports
import numpy, pandas
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [ ]:
### preparing data

### loading heart disease classification data into dataframe
heart_disease = pandas.read_csv(filepath_or_buffer="data-heart-disease.csv")

### splitting data features/target
features = heart_disease.drop(columns="target")
target = heart_disease.loc[:, "target"]

### splitting data train/test
numpy.random.seed(42)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

In [ ]:
### creating random forest classifier

### instantiating model
numpy.random.seed(42)
classifier = RandomForestClassifier(n_estimators=100)

### training model
classifier.fit(X=features_train, y=target_train);

#### Evaluating classification model

--- classification model metrics  
**accuracy:** true predictions / all predictions  
**confusion matrix:** see below  
**classification report:** see below  
**receiver operating characteristic (ROC) curve** see below  
**area under the ROC curve (AUC)** see below

In [ ]:
### imports
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, roc_curve, roc_auc_score

--- `.score()` method  
default `.score()` metric for classification models is accuracy (true predictions / all predictions)

In [ ]:
### evaluating model with model.score() method on training data
classifier.score(X=features_train, y=target_train)

In [ ]:
### evaluating model with model.score() method on test data
classifier.score(X=features_test, y=target_test)

In [ ]:
### evaluating model with default score (accuracy) cross validation ----------------------------------------------------
cv_list = cross_val_score(estimator=classifier, X=features, y=target, cv=5, scoring=None)
cv_mean = numpy.mean(cv_list)
cv_list, cv_mean

--- receiver operating characteristic (ROC) curve  
plots true positive rate (tpr) over false positive rate (fpr)  
visualizes algorithm performance at various algorithm decision thresholds  
suitable for binary classification models  
**true negative rate = specificity:** true negative predictions / all negative targets  
**false positive rate = 1 - specificity:** false positive predictions / all negative targets  
**false negative rate = 1 - sensitivity:** false negative predictions / all positive targets  
**true positive rate = sensitivity = recall:** true positive predictions / all positive targets

--- ROC curve for multiclass classification models  
a ROC curve works with binary output, so multiclass output must be binarized  
one-vs-rest binarization: comparing each class to all the others  
one-vs-one binarization: comparing every pairwise combination of classes


In [ ]:
### function for plotting ROC curve

### function init
def plotRoc(plot_title, tpr, fpr):
    """
    Plots ROC curve, i.e., true positive rate (tpr) over false positive rate (fpr)
    """

    ### plotting ROC curve
    pyplot.plot(fpr, tpr, color="orange", label="ROC Curve")

    ### plotting baseline
    pyplot.plot([0,1], [0,1], color="blue", linestyle="--", label="Guessing")

    ### customizing plot
    pyplot.title(plot_title)
    pyplot.ylabel("True Positive Rate")
    pyplot.xlabel("False Positive Rate")
    pyplot.legend()

    ### rendering plot
    pyplot.show()

    ### function termination
    return

In [ ]:
### evaluating model with ROC curve
target_probs_positive = classifier.predict_proba(X=features_test)[:, 1]
classifier_fpr, classifier_tpr, classifier_thresholds = roc_curve(y_true=target_test, y_score=target_probs_positive)
plotRoc("Receiver Operating Characteristic (ROC)", classifier_tpr, classifier_fpr);

In [ ]:
### perfect ROC curve
perfect_fpr, perfect_tpr, perfect_threshold = roc_curve(y_true=target_test, y_score=target_test)
plotRoc("Perfect ROC", perfect_tpr, perfect_fpr);

In [ ]:
### evaluating model with AUC score ------------------------------------------------------------------------------------
model_auc = roc_auc_score(y_true=target_test, y_score=target_probs_positive)
perfect_auc = roc_auc_score(y_true=target_test, y_score=target_test)
model_auc, perfect_auc

In [ ]:
### evaluating model with confusion matrix -----------------------------------------------------------------------------
target_preds = classifier.predict(X=features_test)
pandas.crosstab(index=target_test, rownames=["Targets"], columns=target_preds, colnames=["Predictions"])

In [ ]:
### visualizing confusion matrix with sklearn --------------------------------------------------------------------------
ConfusionMatrixDisplay.from_predictions(y_true=target_test, y_pred=target_preds);

In [ ]:
### evaluating model with classification report ------------------------------------------------------------------------
print(classification_report(y_true=target_test, y_pred=target_preds))

#### Creating regression model

In [ ]:
### imports
import numpy, pandas
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
### preparing data

### loading california housing dataset
housing_dict = fetch_california_housing()

### creating california housing dataframe
housing_df = pandas.DataFrame(data=housing_dict["data"], columns=housing_dict["feature_names"])
housing_df["MedHouseVal"] = housing_dict["target"]

### splitting data features/target
features = housing_df.drop(columns="MedHouseVal")
target = housing_df.loc[:, "MedHouseVal"]

### splitting data train/test
numpy.random.seed(42)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)
target_test: numpy.ndarray

In [ ]:
### creating random forest regressor

### instantiating model
numpy.random.seed(42)
regressor = RandomForestRegressor(n_estimators=100)

### training model
regressor.fit(X=features_train, y=target_train);

#### Evaluating regression model

In [ ]:
### imports
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

--- `.score()` method  
default `.score()` metric for regression models is [coefficient of determination (r^2)](https://en.wikipedia.org/wiki/Coefficient_of_determination)

In [ ]:
### evaluating model with .score() method on test data
regressor.score(X=features_test, y=target_test)

--- coefficient of determination (r2-score)  
when a model predicts the mean of targets, its r2-score is 0.0  
when a model perfectly predicts all targets, its r2-score is 1.0  
`numpy.full(shape=, fill_value=)` creates an array of `shape=` filled with `fill_value=`

In [ ]:
### getting r2-score of 0.0
target_test_mean = numpy.full(shape=len(target_test), fill_value=target_test.mean())
r2_score(y_true=target_test, y_pred=target_test_mean)

In [ ]:
### getting r2-score of 1.0
r2_score(y_true=target_test, y_pred=target_test)

--- mean absolute error (MAE)  
mean of absolute differences between predictions and targets  
represents the linear magnitude of prediction error

In [ ]:
### computing mean absolute error with sklearn function
target_preds = regressor.predict(X=features_test)
mean_absolute_error(y_true=target_test, y_pred=target_preds)

In [ ]:
### computing mean absolute error step-by-step
error_df = pandas.DataFrame(data={"target preds": target_preds, "target test": target_test})
error_df["differences"] = numpy.abs(error_df["target preds"] - error_df["target test"])
error_df["differences"].mean()

--- mean squared error (MSE)  
mean of squared differences between predictions and targets  
squaring emphasizes large errors and diminishes small errors  
there is also root mean squared error (RMSE) - see sklearn documentation

In [ ]:
### computing mean squared error with sklearn function
mean_squared_error(y_true=target_test, y_pred=target_preds)

In [ ]:
### computing mean squared error step-by-step
error_df["squared differences"] = numpy.square(error_df["differences"])
error_df["squared differences"].mean()